## Productivizar modelo

In [1]:
from flask import Flask, request, jsonify
import pandas as pd
import numpy as np
import sklearn
import pickle
import joblib
import json
import traceback
from sklearn.preprocessing import LabelEncoder

import warnings
warnings.filterwarnings("ignore")

In [2]:
app = Flask(__name__)

In [3]:
# Cargar el codificador y el modelo
label_encoder = joblib.load('label_encoder.pkl')
model = joblib.load('model.pkl')

In [4]:
@app.route('/prediccion', methods=['POST'])
def prediccion():
    try:
        # Obtener los datos de la petición
        data = request.get_json(force=True)
        
        # Convertir los datos de JSON a un DataFrame
        df = pd.json_normalize(data)
        
        # Verificar y procesar la columna 'Timestamp'
        if 'Timestamp' in df.columns:
            df['Timestamp'] = pd.to_datetime(df['Timestamp'], errors='coerce')
            df['Year'] = df['Timestamp'].dt.year
            df['Month'] = df['Timestamp'].dt.month
            df['Day'] = df['Timestamp'].dt.day
            df['Hour'] = df['Timestamp'].dt.hour
            df['Minute'] = df['Timestamp'].dt.minute
            df = df.drop(columns=['Timestamp'])

        # Transformar las columnas categóricas usando el encoder cargado
        categorical_columns = ['Account2', 'Account4', 'Receiving Currency', 'Payment Currency', 'Payment Format']
        for col in categorical_columns:
            if col in df.columns:
                if df[col].dtype == 'object':
                    # Manejar categorías desconocidas
                    df[col] = safe_transform(label_encoder, df[col], col)
                else:
                    # Si la columna no es de tipo 'object', no se transforma
                    df[col] = df[col]

        # Verificar y transformar 'Account4' si es necesario
        if 'Account4' in df.columns:
            try:
                df['Account4'] = df['Account4'].astype(str)  # Convertir a string si es necesario
                df['Account4'] = pd.to_numeric(df['Account4'], errors='coerce')
                df['Account4'].fillna(df['Account4'].mean(), inplace=True)
            except Exception as e:
                print(f"Error al transformar 'Account4': {e}")

        # Convertir a numpy array, excluyendo las columnas que no sean numéricas
        numeric_df = df.select_dtypes(include=[float, int])
        data_to_numpy = numeric_df.values.astype(float)
        
        # Hacer la predicción
        prediction = model.predict(data_to_numpy)
        prediction_list = prediction.tolist()
        
        return jsonify(prediction_list)
    
    except Exception as e:
        print("Error durante la predicción:")
        print(e)
        traceback.print_exc()  # Imprimir el traceback completo del error
        return jsonify({"error": str(e)}), 500

def safe_transform(encoder, series, column_name):
    """ Transformar categorías con un encoder, manejando valores desconocidos. """
    try:
        return encoder.transform(series.astype(str))
    except ValueError:
        # Para categorías desconocidas, asignar un valor por defecto (-1 o similar)
        # Esta parte depende de cómo has entrenado tu encoder. 
        # Asegúrate de ajustar esto según el codificador específico.
        return pd.Series([encoder.transform([x])[0] if x in encoder.classes_ else -1 for x in series])

In [5]:
if __name__ == '__main__':
    app.run(host='127.0.0.1', port=8080)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8080
Press CTRL+C to quit
127.0.0.1 - - [26/Aug/2024 18:22:11] "POST /prediccion HTTP/1.1" 200 -
127.0.0.1 - - [26/Aug/2024 18:22:14] "POST /prediccion HTTP/1.1" 200 -


In [4]:
from flask import Flask, request, jsonify
import pandas as pd
import traceback

app = Flask(__name__)

# Aquí deberías cargar tu modelo y el codificador
# Supongamos que ya están cargados en las variables `model` y `label_encoder`

@app.route('/prediccion', methods=['POST'])
def prediccion():
    try:
        # Obtener los datos de la petición
        data = request.get_json(force=True)
        
        # Convertir los datos de JSON a un DataFrame
        df = pd.json_normalize(data)
        
        # Verificar y procesar la columna 'Timestamp'
        if 'Timestamp' in df.columns:
            df['Timestamp'] = pd.to_datetime(df['Timestamp'], errors='coerce')
            df['Year'] = df['Timestamp'].dt.year
            df['Month'] = df['Timestamp'].dt.month
            df['Day'] = df['Timestamp'].dt.day
            df['Hour'] = df['Timestamp'].dt.hour
            df['Minute'] = df['Timestamp'].dt.minute
            df = df.drop(columns=['Timestamp'])

        # Transformar las columnas categóricas usando el encoder cargado
        categorical_columns = ['Account2', 'Account4', 'Receiving Currency', 'Payment Currency', 'Payment Format']
        for col in categorical_columns:
            if col in df.columns:
                if df[col].dtype == 'object':
                    # Manejar categorías desconocidas
                    df[col] = safe_transform(label_encoder, df[col], col)
                else:
                    # Si la columna no es de tipo 'object', no se transforma
                    df[col] = df[col]

        # Verificar y transformar 'Account4' si es necesario
        if 'Account4' in df.columns:
            try:
                df['Account4'] = df['Account4'].astype(str)  # Convertir a string si es necesario
                df['Account4'] = pd.to_numeric(df['Account4'], errors='coerce')
                df['Account4'].fillna(df['Account4'].mean(), inplace=True)
            except Exception as e:
                print(f"Error al transformar 'Account4': {e}")

        # Convertir a numpy array, excluyendo las columnas que no sean numéricas
        numeric_df = df.select_dtypes(include=[float, int])
        data_to_numpy = numeric_df.values.astype(float)
        
        # Hacer la predicción
        prediction = model.predict(data_to_numpy)
        prediction_proba = model.predict_proba(data_to_numpy)
        
        # Convertir predicciones y probabilidades a listas para el JSON de respuesta
        prediction_list = prediction.tolist()
        prediction_proba_list = prediction_proba.tolist()
        
        # Devolver tanto la predicción como la probabilidad en la respuesta
        return jsonify({
            "prediction": prediction_list,
            "probability": prediction_proba_list
        })
    
    except Exception as e:
        print("Error durante la predicción:")
        print(e)
        traceback.print_exc()  # Imprimir el traceback completo del error
        return jsonify({"error": str(e)}), 500

def safe_transform(encoder, series, column_name):
    """ Transformar categorías con un encoder, manejando valores desconocidos. """
    try:
        return encoder.transform(series.astype(str))
    except ValueError:
        # Para categorías desconocidas, asignar un valor por defecto (-1 o similar)
        # Esta parte depende de cómo has entrenado tu encoder. 
        # Asegúrate de ajustar esto según el codificador específico.
        return pd.Series([encoder.transform([x])[0] if x in encoder.classes_ else -1 for x in series])


In [ ]:
if __name__ == '__main__':
    app.run(host='127.0.0.1', port=8080)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8080
Press CTRL+C to quit
127.0.0.1 - - [26/Aug/2024 20:05:48] "POST /prediccion HTTP/1.1" 200 -
